In [3]:
using Polymake

polymake version 4.11
Copyright (c) 1997-2023
Ewgenij Gawrilow, Michael Joswig, and the polymake team
Technische Universität Berlin, Germany
https://polymake.org

This is free software licensed under GPL; see the source for copying conditions.
There is NO warranty; not even for MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.



In [64]:
p = polytope.Polytope(POINTS=[1 -1 -1; 1 1 -1; 1 -1 1; 1 1 1; 1 0 0])
@show p.VERTICES

p.VERTICES = pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1



pm::Matrix<pm::Rational>
1 -1 -1
1 1 -1
1 -1 1
1 1 1


In [32]:
MAX_LATTICE_POINTS = 12 #global variable for the maximum number of lattice points
polytopes_list = [] #global list of polytopes
open("SmoothGeneration_output/SmoothGeneration_output4") do f #open the file
    number_polytopes = 0
    polytopes_read = 0
    while ! eof(f)
        #Read the number of vertices
        number_vertices = parse(Int, readline(f))
        polytopes_read += 1

        
        vertices_matrix = zeros(Int, number_vertices, 4)
        #Read the matrix of vertices of polytopes P
        for a in 1:number_vertices
            vertices_string = readline(f)
            vertex_vector = [1;parse.(Int64, split(vertices_string))] #append 1 to the beginning due to polymake standards
            vertices_matrix[a, :] .= vertex_vector
        end
        #Create the polytope from its vertices
        new_polytope = polytope.Polytope(POINTS=vertices_matrix)
        
        if new_polytope.SMOOTH #Checks if the polytope is smooth
            if new_polytope.N_LATTICE_POINTS <= MAX_LATTICE_POINTS #Check if the polytope does not exceed the maximum number of lattice points
                isomorphic_found = false
                for smooth_polytope in polytopes_list
                    if polytope.lattice_isomorphic_smooth_polytopes(smooth_polytope, new_polytope) #Check if any of the polytopes are lattice-equivalent
                        isomorphic_found = true
                        #println("Isomorphic lattice polytope found!")
                        break
                    end
                end
                if !isomorphic_found
                    push!(polytopes_list, new_polytope)
                    number_polytopes += 1
                end
            end
        else
            println("Non-smooth polytope found!")
        end
    end 
    println(number_polytopes, " smooth polytopes extracted of ", polytopes_read, " polytopes read from file")
end

output_file_prefix = "Prune_output/prune_output"

for smooth_polytope in polytopes_list
    println(typeof(smooth_polytope.N_LATTICE_POINTS))
    num_vertices_str = string(UInt(smooth_polytope.N_LATTICE_POINTS))
    output_file = "$output_file_prefix$num_vertices_str"
    if !isfile(output_file)
        touch(output_file)  # Create an empty file if it doesn't exist
    end
    open(output_file, "a") do file #Open the file in appending mode
        println(file, smooth_polytope.N_VERTICES)

        #Printing the contents of the matrix and avoiding the Polymake types showing up
        for row in eachrow(smooth_polytope.VERTICES) 
            println(file, join(row[2:end], " "))
        end
    end
end

2 smooth polytopes found of 4 polytopes read from file
Polymake.LibPolymake.IntegerAllocated
Polymake.LibPolymake.IntegerAllocated


In [19]:
println(number_polytopes)


0
